In [1]:
import pandas as pd
import numpy as np
#import spark for python
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import min, max, sum, count, mean, avg
from pyspark.sql.functions import concat, lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import month, year, quarter, round
#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

from pydataset import data

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 14:17:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1.
Create a spark data frame that contains your favorite programming languages.
The name of the column should be language


In [2]:
# Define your favorite programming languages
favorite_languages = ["Python", "Java", "JavaScript", "Ruby", "Go", "R"]

# Create a pandas DataFrame with a column named "language"
pandas_dataframe = pd.DataFrame({"language": favorite_languages})

# Display the DataFrame
pandas_dataframe


,language
0,Python
1,Java
2,JavaScript
3,Ruby
4,Go
5,R


In [3]:
#create spark dataframe 
df = spark.createDataFrame(pandas_dataframe)
df.show()

+----------+
|  language|
+----------+
|    Python|
|      Java|
|JavaScript|
|      Ruby|
|        Go|
|         R|
+----------+



View the schema of the dataframe


In [4]:
#print schema
df.printSchema()

root
 |-- language: string (nullable = true)



Output the shape of the dataframe


In [5]:
#show shape (spark)
df.count(), len(df.columns)


(6, 1)

Show the first 5 records in the dataframe

In [6]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|JavaScript|
|      Ruby|
|        Go|
+----------+
only showing top 5 rows



# 2.
Load the mpg dataset as a spark dataframe.




In [7]:
#import mpg as a spark data frame
df = spark.createDataFrame(data('mpg'))
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

Create 1 column of output that contains a message like the one below:

The 1999 audi a4 has a 4 cylinder engine.

In [8]:
# Select the desired columns and create a new column with the desired output message
df = df.select(
    concat(
        lit("The "), df["year"], lit(" "), df["manufacturer"], lit(" "), df["model"],
        lit(" has a "), df["cyl"], lit(" cylinder engine.")
    ).alias("output")
)

df.show(1, truncate=False)



+-----------------------------------------+
|output                                   |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
+-----------------------------------------+
only showing top 1 row



For each vehicle.

Transform the trans column so that it only contains either manual or auto.

In [9]:
# get clean df
df = spark.createDataFrame(data('mpg'))
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [10]:
# Transform the "trans" column
df = df.withColumn("trans", when(df["trans"].like("%manual%"), "manual").otherwise("auto"))

# Show the updated DataFrame
df.show()

+------------+------------------+-----+----+---+------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto|  4| 16| 25|  p|compact|
|        audi|        a4 quattro|  2.0|2

# 3.
Load the tips dataset as a spark dataframe.




In [11]:
# get clean df
df = spark.createDataFrame(data('tips'))
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

What percentage of observations are smokers?

In [30]:
smoker_percentage = df.groupBy().agg((count(when(df["smoker"] == "Yes", 1)) / count("*")) * 100)
smoker_percentage.show()

+---------------------------------------------------------------+
|((count(CASE WHEN (smoker = Yes) THEN 1 END) / count(1)) * 100)|
+---------------------------------------------------------------+
|                                             38.114754098360656|
+---------------------------------------------------------------+



Create a column that contains the tip percentage


In [31]:
# Create a new column with the tip percentage
df = df.withColumn("tip_percentage", (col("tip") / col("total_bill")) * 100)

# Show the updated DataFrame
df.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|    tip_percentage|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

Calculate the average tip percentage for each combination of sex and smoker.

In [32]:
# Calculate the average tip percentage for each combination of sex and smoker
avg_tip_percentage = df.groupBy("sex", "smoker").agg(avg("tip_percentage").alias("avg_tip_percentage"))

# Show the DataFrame
avg_tip_percentage.show()

+------+------+------------------+
|   sex|smoker|avg_tip_percentage|
+------+------+------------------+
|  Male|    No| 16.06687151291298|
|Female|    No| 15.69209707691836|
|  Male|   Yes|15.277117520248513|
|Female|   Yes| 18.21503526994103|
+------+------+------------------+



# 4. 
Use the seattle weather dataset referenced in the lesson to answer the questions below.



In [33]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
df = spark.createDataFrame(weather)
df.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



Convert the temperatures to fahrenheit.






In [34]:
# Convert temperatures from Celsius to Fahrenheit
df = df.withColumn("temp_max_fahrenheit", (df["temp_max"] * 9/5) + 32)
df = df.withColumn("temp_min_fahrenheit", (df["temp_min"] * 9/5) + 32)

# Show the updated DataFrame
df.show()

+----------+-------------+--------+--------+----+-------+-------------------+-------------------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_fahrenheit|temp_min_fahrenheit|
+----------+-------------+--------+--------+----+-------+-------------------+-------------------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|              55.04|               41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|              51.08|              37.04|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|              53.06|              44.96|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|              53.96|              42.08|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|              48.02|              37.04|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|              39.92|              35.96|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|              44.96|              37.04|
|2012-01-08|        


Which month has the most rain, on average?


In [35]:
# Extract the month from the date column
df = df.withColumn("month", month(df["date"]))

# Calculate the average rainfall for each month
avg_rainfall_by_month = df.groupBy("month").agg(avg("precipitation").alias("avg_rainfall"))

# Find the month with the highest average rainfall
month_with_most_rain = avg_rainfall_by_month.orderBy("avg_rainfall", ascending=False).first()["month"]

Month with the most rain, on average: 11


In [39]:
avg_rainfall_by_month.show()

+-----+-------------------+
|month|       avg_rainfall|
+-----+-------------------+
|    1| 3.7580645161290316|
|    6| 1.1075000000000002|
|    3|  4.888709677419355|
|    5| 1.6733870967741935|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|   12|  5.021774193548389|
|    9| 1.9624999999999997|
|    8| 1.3201612903225806|
|    7|0.38870967741935486|
|   10|  4.059677419354839|
|   11|  5.354166666666667|
+-----+-------------------+



In [37]:
month_with_most_rain

11


Which year was the windiest?


In [42]:
# Extract the year from the date column
df = df.withColumn("year", year(df["date"]))

# Calculate the average wind speed for each year
avg_wind_speed_by_year = df.groupBy("year").agg(avg("wind").alias("avg_wind_speed"))

# Find the year with the highest average wind speed
year_with_highest_wind = avg_wind_speed_by_year.orderBy("avg_wind_speed", ascending=False).first()["year"]




In [43]:
avg_wind_speed_by_year.show()

+----+------------------+
|year|    avg_wind_speed|
+----+------------------+
|2012| 3.400819672131148|
|2013|3.0158904109589058|
|2014| 3.387671232876714|
|2015| 3.159726027397261|
+----+------------------+



In [55]:
print('The windiest year was',year_with_highest_wind )

The windiest year was 2012


What is the most frequent type of weather in January?


In [56]:
# Filter the data for the month of January
january_data = df.filter(month(df["date"]) == 1)
january_data.show()

+----------+-------------+--------+--------+----+-------+-------------------+-------------------+-----+----+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_fahrenheit|temp_min_fahrenheit|month|year|
+----------+-------------+--------+--------+----+-------+-------------------+-------------------+-----+----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|              55.04|               41.0|    1|2012|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|              51.08|              37.04|    1|2012|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|              53.06|              44.96|    1|2012|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|              53.96|              42.08|    1|2012|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|              48.02|              37.04|    1|2012|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|              39.92|              35.96|    1|2012|
|2012-01-07|       

In [57]:
# Calculate the mode of the weather column in January
most_frequent_weather_january = january_data.groupBy("weather").count().orderBy(col("count").desc()).first()["weather"]
print("January's most frequent weather type is",most_frequent_weather_january)

January's most frequent weather type is fog


What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [58]:
# Filter the data for July in the years 2013 and 2014 and sunny weather
filtered_data = df.filter((month(df["date"]) == 7) & ((year(df["date"]) == 2013) | (year(df["date"]) == 2014)) & (df["weather"] == "sunny"))

In [59]:
# Calculate the average high and low temperatures
avg_temperatures = filtered_data.groupBy().agg(avg("temp_max").alias("avg_high_temp"), avg("temp_min").alias("avg_low_temp")).collect()[0]

In [60]:
# Retrieve the average high and low temperatures
avg_high_temp = avg_temperatures["avg_high_temp"]
avg_low_temp = avg_temperatures["avg_low_temp"]
print("Average High Temperature on Sunny Days in July (2013-2014): {}".format(avg_high_temp))
print("Average Low Temperature on Sunny Days in July (2013-2014): {}".format(avg_low_temp))

Average High Temperature on Sunny Days in July (2013-2014): None
Average Low Temperature on Sunny Days in July (2013-2014): None


What percentage of days were rainy in q3 of 2015?


In [61]:
# Filter the data for Q3 of 2015 and rainy weather
q3_2015_rainy_days = df.filter((quarter(df["date"]) == 3) & (year(df["date"]) == 2015) & (df["weather"] == "rain"))

In [62]:
# Calculate the total number of days in Q3 of 2015
total_days_q3_2015 = q3_2015_rainy_days.count()

In [63]:
# Calculate the number of rainy days in Q3 of 2015
rainy_days_q3_2015 = q3_2015_rainy_days.filter(df["weather"] == "rain").count()

In [64]:

# Calculate the percentage of rainy days in Q3 of 2015
percentage_rainy_days_q3_2015 = (rainy_days_q3_2015 / total_days_q3_2015) * 100

In [65]:
# Print the percentage of rainy days in Q3 of 2015
print("Percentage of Rainy Days in Q3 of 2015: {:.2f}%".format(percentage_rainy_days_q3_2015))

Percentage of Rainy Days in Q3 of 2015: 100.00%


For each year, find what percentage of days it rained (had non-zero precipitation).

In [72]:
# Calculate the total count of days and count of rainy days for each year
yearly_counts = df.groupBy(year(df["date"]).alias("year")).agg(count("*").alias("total_days"),
                                                                  sum(when(df["precipitation"] > 0, 1).otherwise(0)).alias("rainy_days"))

In [73]:
# Calculate the percentage of rainy days for each year
yearly_percentages = yearly_counts.withColumn("percentage_rainy_days", (yearly_counts["rainy_days"] / yearly_counts["total_days"]) * 100)

In [74]:
# Show the resulting DataFrame
yearly_percentages.show()

+----+----------+----------+---------------------+
|year|total_days|rainy_days|percentage_rainy_days|
+----+----------+----------+---------------------+
|2012|       366|       177|    48.36065573770492|
|2013|       365|       152|    41.64383561643836|
|2014|       365|       150|     41.0958904109589|
|2015|       365|       144|    39.45205479452055|
+----+----------+----------+---------------------+

